# source_browser

> Source browser components for displaying and filtering transcription sources

In [ ]:
#| default_exp components.source_browser

In [ ]:
#| export
import json
from typing import Any, Dict, List

from fasthtml.common import (
    Div, Span, Input, Button, Label, Table, Thead, Tbody, Tr, Th, Td, Hidden, Select, Option
)

# DaisyUI components
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.data_display.table import table, table_modifiers
from cjm_fasthtml_daisyui.components.data_input.checkbox import checkbox, checkbox_sizes
from cjm_fasthtml_daisyui.components.data_input.select import select, select_sizes
from cjm_fasthtml_daisyui.components.data_input.text_input import text_input, text_input_sizes
from cjm_fasthtml_daisyui.components.feedback.tooltip import (
    tooltip, tooltip_content, tooltip_placement
)
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, min_h, max_w
from cjm_fasthtml_tailwind.utilities.typography import (
    font_size, font_weight, font_family, uppercase, tracking, truncate, whitespace
)
from cjm_fasthtml_tailwind.utilities.layout import overflow, display_tw
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.utilities.effects import shadow
from cjm_fasthtml_tailwind.utilities.interactivity import cursor
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, gap, grow
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Local imports
from cjm_transcript_source_select.html_ids import SelectionHtmlIds
from cjm_transcript_source_select.utils import (
    count_words, format_date, format_audio_filename
)
from cjm_transcript_source_select.services.source_utils import (
    extract_batch_id, group_transcriptions, is_source_selected,
    check_audio_exists, extract_model_name
)

## Grouping Selector

In [ ]:
#| export
def _render_grouping_selector(
    grouping_mode: str,  # Current grouping mode: "media_path" or "batch_id"
    grouping_change_url: str,  # URL for changing grouping mode
) -> Any:  # Grouping selector component
    """Render the dropdown for selecting grouping mode."""
    return Div(
        Label(
            "Group by:",
            cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70), m.r(2))
        ),
        Select(
            Option("Audio File", value="media_path", selected=grouping_mode == "media_path"),
            Option("Batch ID", value="batch_id", selected=grouping_mode == "batch_id"),
            name="grouping_mode",
            id=SelectionHtmlIds.GROUPING_SELECTOR,
            cls=combine_classes(select, select_sizes.sm),
            hx_post=grouping_change_url,
            hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_LIST),
            hx_swap="outerHTML"
        ),
        cls=combine_classes(flex_display, items.center)
    )

## Source Row

In [ ]:
#| export
def _render_source_row(
    record: Dict[str, Any],  # Transcription record
    is_selected: bool,  # Whether this source is selected
    add_url: str,  # URL for adding to queue
    remove_url: str,  # URL for removing from queue
    preview_url: str,  # URL for previewing content
    is_first: bool = False,  # Whether this is the first row (gets initial focus)
    row_index: int = 0,  # Index among selectable rows (for keyboard focus sync)
) -> Any:  # Table row element
    """Render a single source row in the browser table."""
    record_id = record.get("record_id", "")
    provider_id = record.get("provider_id", "")
    text = record.get("text", "")
    created_at = record.get("created_at", "")
    media_path = record.get("media_path", "")
    metadata = record.get("metadata")
    word_count = count_words(text)
    
    # Extract model name from metadata
    model_name = extract_model_name(metadata)
    
    # Check if audio file exists
    audio_exists = check_audio_exists(media_path)
    
    # Determine action URL based on selection state
    action_url = remove_url if is_selected else add_url
    
    # Model badge with tooltip showing plugin name
    model_badge = Div(
        Div(
            Div(
                f"Plugin: {provider_id}",
                cls=combine_classes(whitespace.normal, max_w.xs)
            ),
            cls=str(tooltip_content)
        ),
        Span(
            model_name,
            cls=combine_classes(badge, badge_colors.neutral, badge_sizes.xs, whitespace.nowrap)
        ),
        cls=combine_classes(tooltip, tooltip_placement.top)
    )
    
    # Audio badge with tooltip showing full path
    audio_tooltip_text = media_path if media_path else "No audio path"
    audio_badge = Div(
        Div(
            Div(
                audio_tooltip_text,
                cls=combine_classes(whitespace.normal, max_w.xs)
            ),
            cls=str(tooltip_content)
        ),
        Span(
            "Yes" if audio_exists else "No",
            cls=combine_classes(
                badge, badge_sizes.xs, whitespace.nowrap,
                badge_colors.success if audio_exists else badge_colors.error
            )
        ),
        cls=combine_classes(tooltip, tooltip_placement.top)
    )
    
    # Click handler to sync keyboard focus with clicked row
    zone_id = SelectionHtmlIds.SOURCE_LIST
    focus_onclick = f"if(window.kbNav)window.kbNav.setItemFocus('{zone_id}',{row_index},true)"
    
    return Tr(
        # Hidden cell for keyboard action data (must be in TD for valid HTML)
        Td(
            Hidden(name="record_id", value=record_id),
            Hidden(name="provider_id", value=provider_id),
            cls=str(display_tw.hidden)
        ),
        Td(
            Input(
                type="checkbox",
                checked="checked" if is_selected else None,
                cls=combine_classes(checkbox, checkbox_sizes.sm),
                hx_post=action_url,
                hx_vals=json.dumps({"record_id": record_id, "provider_id": provider_id}),
                hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
                hx_swap="outerHTML",
                name=f"source_{provider_id}_{record_id}"
            ),
            cls=str(w(12))
        ),
        Td(
            Span(record_id[:12] + "..." if len(record_id) > 12 else record_id, 
                 cls=combine_classes(font_size.xs, font_family.mono)),
            title=record_id
        ),
        Td(model_badge),
        Td(
            format_date(created_at),
            cls=combine_classes(font_size.xs, text_dui.base_content.opacity(60))
        ),
        Td(audio_badge),
        Td(
            f"{word_count:,} words",
            cls=str(font_size.xs)
        ),
        id=SelectionHtmlIds.source_row(record_id, provider_id),
        cls=combine_classes(
            bg_dui.primary.opacity(10) if is_selected else "",
            bg_dui.base_200.hover,
            cursor.pointer
        ),
        tabindex="0",
        data_selectable="true",
        data_focused="true" if is_first else "false",
        data_record_id=record_id,
        data_provider_id=provider_id,
        onclick=focus_onclick,
        hx_get=preview_url,
        hx_vals=json.dumps({"record_id": record_id, "provider_id": provider_id}),
        hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.PREVIEW_PANEL),
        hx_swap="outerHTML",
        hx_trigger="click"
    )

## Group Header

In [ ]:
#| export
def _render_group_header(
    group_key: str,  # The group key (media_path or batch_id value)
    record_count: int,  # Number of records in this group
    select_all_url: str,  # URL for selecting all in group
    grouping_mode: str = "media_path",  # Current grouping mode
) -> Any:  # Table row for group header
    """Render a group header row."""
    # Format display text based on grouping mode
    if grouping_mode == "batch_id":
        display_text = group_key  # batch_id is already formatted
    else:
        # media_path - extract filename
        display_text = format_audio_filename(group_key)
    
    return Tr(
        Td(
            Div(
                Span(display_text, cls=str(font_weight.bold)),
                Button(
                    f"Select All {record_count}",
                    cls=combine_classes(btn, btn_styles.ghost, btn_sizes.xs),
                    hx_post=select_all_url,
                    hx_vals=json.dumps({"group_key": group_key, "grouping_mode": grouping_mode}),
                    hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
                    hx_swap="outerHTML"
                ) if record_count > 1 else None,
                cls=combine_classes(flex_display, justify.between, items.center, p.r(4))
            ),
            colspan="7",  # Updated for hidden column
            cls=combine_classes(bg_dui.base_200.opacity(50), p.y(2), font_size.xs, uppercase, tracking.wide)
        )
    )

In [ ]:
#| export
def _render_audio_group_header(
    media_path: str,  # Path to audio file
    record_count: int,  # Number of records in this group
    select_all_url: str,  # URL for selecting all in group
) -> Any:  # Table row for group header
    """Render a group header row for an audio file (legacy wrapper)."""
    return _render_group_header(media_path, record_count, select_all_url, grouping_mode="media_path")

## Source List

In [ ]:
#| export
def _render_source_list(
    transcriptions: List[Dict[str, Any]],  # Available transcription records
    selected_sources: List[Dict[str, str]],  # Currently selected sources
    add_url: str,  # URL for adding to queue
    remove_url: str,  # URL for removing from queue
    preview_url: str,  # URL for previewing content
    select_all_url: str,  # URL for selecting all in a group
    grouping_mode: str = "media_path",  # Grouping mode: "media_path" or "batch_id"
    oob: bool = False,  # Whether to include hx-swap-oob for out-of-band swap
) -> Any:  # Source list container with table
    """Render the source list table with grouped rows."""
    # Group transcriptions by the specified mode
    grouped = group_transcriptions(transcriptions, group_by=grouping_mode)
    
    # Build table rows
    table_rows = []
    row_index = 0
    for group_key, records in grouped.items():
        # Add group header
        table_rows.append(_render_group_header(
            group_key, len(records), select_all_url, grouping_mode
        ))
        
        # Add record rows
        for record in records:
            is_selected = is_source_selected(
                record.get("record_id", ""), record.get("provider_id", ""), selected_sources
            )
            table_rows.append(_render_source_row(
                record, is_selected, add_url, remove_url, preview_url,
                is_first=(row_index == 0),
                row_index=row_index,
            ))
            row_index += 1
    
    return Div(
        Table(
            Thead(
                Tr(
                    Th("", cls=str(display_tw.hidden)),  # Hidden column for keyboard data
                    Th("", cls=str(w(12))),
                    Th("Job ID"),
                    Th("Model"),
                    Th("Date"),
                    Th("Audio"),
                    Th("Length")
                )
            ),
            Tbody(*table_rows),
            cls=combine_classes(table, table_modifiers.pin_rows)
        ),
        id=SelectionHtmlIds.SOURCE_LIST,
        cls=combine_classes(grow(), overflow.y.auto, border_radius.box),
        hx_swap_oob="outerHTML" if oob else None
    )

## Source Browser

In [ ]:
#| export
def _render_source_browser(
    transcriptions: List[Dict[str, Any]],  # Available transcription records
    sources: List[Dict[str, Any]],  # Available source plugins (unused, kept for API compat)
    selected_sources: List[Dict[str, str]],  # Currently selected sources
    add_url: str,  # URL for adding to queue
    remove_url: str,  # URL for removing from queue
    preview_url: str,  # URL for previewing content
    select_all_url: str,  # URL for selecting all in a group
    filter_url: str,  # URL for filtering sources
    grouping_mode: str = "media_path",  # Current grouping mode
    grouping_change_url: str = "",  # URL for changing grouping mode
) -> Any:  # Source browser component
    """Render the source browser panel with search filtering and grouped table."""
    # Render the source list table
    source_list = _render_source_list(
        transcriptions=transcriptions,
        selected_sources=selected_sources,
        add_url=add_url,
        remove_url=remove_url,
        preview_url=preview_url,
        select_all_url=select_all_url,
        grouping_mode=grouping_mode
    )
    
    return Div(
        # Header bar with search and grouping selector
        Div(
            # Search input
            Input(
                type="search",
                placeholder="Search by filename, job ID, or text...",
                cls=combine_classes(text_input, text_input_sizes.sm, grow()),
                id=SelectionHtmlIds.SOURCE_FILTER_SEARCH,
                name="search",
                hx_get=filter_url,
                hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_LIST),
                hx_trigger="input changed delay:500ms, keyup[key=='Enter']",
                hx_swap="outerHTML",
                onkeydown="if(event.key==='Enter') event.preventDefault()"
            ),
            # Grouping selector
            _render_grouping_selector(grouping_mode, grouping_change_url) if grouping_change_url else None,
            cls=combine_classes(
                p(4), border_dui.base_200, border.b(),
                flex_display, gap(4), items.center
            )
        ),
        
        # Table
        source_list,
        
        id=SelectionHtmlIds.SOURCE_BROWSER,
        cls=combine_classes(
            # Sizing
            w.full,
            grow(),
            min_h(64),
            # Base styling
            bg_dui.base_100,
            border_radius.box,
            shadow.lg,
            border_dui.base_300,
            flex_display,
            flex_direction.col
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()